In [1]:
from src.get_modelnet40.load_data import get_dls_for_viz
from src.it_net.it_net import ITNet
import torch

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pyvista as pv
from tqdm import tqdm
import scipy

In [2]:
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import random

In [164]:
min_xyz = [-2,-2,-2]
max_xyz = [2,2,2]

In [170]:
def visualize_rotate(data):
    # x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    x_eye, y_eye, z_eye = 1.25, 1.25, 1.25
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze)),
            # xaxis=dict(range=[-1, 1]),
            # yaxis=dict(range=[-1, 1]),
            # zaxis=dict(range=[-1, 1])
        ))))
    fig = go.Figure(data=data,
        layout=go.Layout(
            updatemenus=[dict(type='buttons',
                showactive=False,
                y=1,
                x=0.8,
                xanchor='left',
                yanchor='bottom',
                pad=dict(t=45, r=10),
                buttons=[dict(label='Play',
                    method='animate',
                    args=[None, dict(frame=dict(duration=50, redraw=True),
                        transition=dict(duration=0),
                        fromcurrent=True,
                        mode='immediate'
                        )]
                    )
                ])]
        ),
        frames=frames
    )

    return fig


def pcshow_double(xs_list, ys_list, zs_list, min_xyz, max_xyz):
    # Create subplots with 1 row and 2 columns
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]])

    # Iterate over the provided point clouds
    for i, (xs, ys, zs) in enumerate(zip(xs_list, ys_list, zs_list), start=1):
        # Add Scatter3d trace for each point cloud to respective subplot
        fig.add_trace(go.Scatter3d(x=xs, y=ys, z=zs, mode='markers', 
                                    name=f'Point Cloud {i}', marker=dict(size=2)), row=1, col=i)

    # Update layout with axis ranges
    fig.update_layout(scene=dict(aspectmode='data',
                                 aspectratio=dict(x=1, y=1, z=1),
                                 xaxis=dict(range=[min_xyz[0], max_xyz[0]]),
                                 yaxis=dict(range=[min_xyz[1], max_xyz[1]]),
                                 zaxis=dict(range=[min_xyz[2], max_xyz[2]])))
    
    # Show the figure
    fig.show()

def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    
    # fig.update_layout(scene=dict(aspectmode='data',
    #                              aspectratio=dict(x=1, y=1, z=1),
    #                              xaxis=dict(range=[min_xyz[0], max_xyz[0]]),
    #                              yaxis=dict(range=[min_xyz[1], max_xyz[1]]),
    #                              zaxis=dict(range=[min_xyz[2], max_xyz[2]])))
    fig.update_layout(scene=dict(aspectmode='cube',
                                 aspectratio=dict(x=1, y=1, z=1)))
    
    fig.show()

In [5]:
class ElasticDistortion:
    """Apply elastic distortion on sparse coordinate space. First projects the position onto a 
    voxel grid and then apply the distortion to the voxel grid.

    Parameters
    ----------
    granularity: List[float]
        Granularity of the noise in meters
    magnitude:List[float]
        Noise multiplier in meters
    Returns
    -------
    data: Data
        Returns the same data object with distorted grid
    """

    def __init__(
        self, apply_distorsion: bool = True, granularity=[0.2, 0.8], magnitude=[0.4, 1.6],
    ):
        assert len(magnitude) == len(granularity)
        self._apply_distorsion = apply_distorsion
        self._granularity = granularity
        self._magnitude = magnitude

    @staticmethod
    def elastic_distortion(coords, granularity, magnitude):
        coords = coords.numpy()
        blurx = np.ones((3, 1, 1, 1)).astype("float32") / 3
        blury = np.ones((1, 3, 1, 1)).astype("float32") / 3
        blurz = np.ones((1, 1, 3, 1)).astype("float32") / 3
        coords_min = coords.min(0)

        # Create Gaussian noise tensor of the size given by granularity.
        noise_dim = ((coords - coords_min).max(0) // granularity).astype(int) + 3
        noise = np.random.randn(*noise_dim, 3).astype(np.float32)

        # Smoothing.
        for _ in range(2):
            noise = scipy.ndimage.filters.convolve(noise, blurx, mode="constant", cval=0)
            noise = scipy.ndimage.filters.convolve(noise, blury, mode="constant", cval=0)
            noise = scipy.ndimage.filters.convolve(noise, blurz, mode="constant", cval=0)

        # Trilinear interpolate noise filters for each spatial dimensions.
        ax = [
            np.linspace(d_min, d_max, d)
            for d_min, d_max, d in zip(coords_min - granularity, coords_min + granularity * (noise_dim - 2), noise_dim)
        ]
        interp = scipy.interpolate.RegularGridInterpolator(ax, noise, bounds_error=0, fill_value=0)
        coords = coords + interp(coords) * magnitude
        return torch.tensor(coords).float()

    def __call__(self, data):
        # coords = data.pos / self._spatial_resolution
        if self._apply_distorsion:
            if random.random() < 0.95:
                for i in range(len(self._granularity)):
                    data = ElasticDistortion.elastic_distortion(data, self._granularity[i], self._magnitude[i],)
        return data

    def __repr__(self):
        return "{}(apply_distorsion={}, granularity={}, magnitude={})".format(
            self.__class__.__name__, self._apply_distorsion, self._granularity, self._magnitude,
        )

In [27]:
device = "mps"
batch_size = 128

In [28]:
train_loader = get_dls_for_viz(batch_size=batch_size)
batch = next(iter(train_loader))

In [29]:
batch_point_clouds = batch["pointcloud"].to(torch.float32).to(device)
# batch_point_clouds = batch_point_clouds.transpose(1, 2)

In [30]:
# torch_input = batch_point_clouds[8,:,:].transpose(0, 1).detach().cpu()
# cloud = torch_input.numpy()
# cloud = cloud @ np.array([[1.25, 0, 0], [0, 1.25, 0], [0, 0, 1.25]])
# x1 = cloud[:,0]
# y1 = cloud[:,1]
# z1 = cloud[:,2]

# # ed = ElasticDistortion()
# # cloud = ed(torch_input)
# # cloud = cloud @ np.array([[0.75, 0, 0], [0, 0.75, 0], [0, 0, 0.75]])
# cloud = cloud[:512,:]
# x2 = cloud[:,0]
# y2 = cloud[:,1]
# z2 = cloud[:,2]

# pcshow([x1,x2], [y1,y2], [z1,z2], min_xyz, max_xyz)

In [31]:
# ed = ElasticDistortion()
# print(ed(batch_point_clouds.transpose(1,2).detach().cpu()[0,:,:]))

In [199]:
from src.pretrain_utils.corruptions import *
import copy

print("batch_point_clouds", batch_point_clouds.shape)

corruptions = [Jitter(), Scale(same=True, min_scale=0.5, max_scale=2), ElasticDistortion(), Dropout(p=0.5)]
corrupted = copy.deepcopy(batch_point_clouds)
for corruption in corruptions:
    corrupted = corruption(corrupted)

print("corrupted", corrupted.shape)

index = 33
cloud1 = batch_point_clouds[index,:,:].transpose(0, 1).detach().cpu().numpy()
cloud2 = corrupted[index,:,:].transpose(0, 1).detach().cpu().numpy()

batch_point_clouds torch.Size([128, 1024, 3])
corrupted torch.Size([128, 1024, 3])


In [200]:
pcshow(*cloud1)

In [201]:
pcshow(*cloud2)